In [1]:
# ▶ 1. 필수 라이브러리
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import pickle
import warnings
warnings.filterwarnings('ignore')

# ▶ 2. 데이터 로딩
df = pd.read_csv('피처_40/train_40.csv', encoding='utf-8-sig')

# ▶ 3. y_vif 생성 (Segment → 숫자 라벨로)
le = LabelEncoder()
y_vif = le.fit_transform(df['Segment'])

X_vif = df.drop(columns=['Segment', 'ID'])
cat_features = X_vif.select_dtypes(include='object').columns.tolist()

In [ ]:
# ▶ 4. 교차검증 (폴드별 F1_micro 출력)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_scores = []

for fold, (train_idx, valid_idx) in enumerate(cv.split(X_vif, y_vif), 1):
    X_train, X_valid = X_vif.iloc[train_idx], X_vif.iloc[valid_idx]
    y_train, y_valid = y_vif[train_idx], y_vif[valid_idx]
    
    model = CatBoostClassifier(
        loss_function="MultiClass",
        eval_metric="MultiClass",
        task_type="GPU",
        learning_rate=0.01,
        iterations=70000,
        l2_leaf_reg=50,
        random_seed=42,
        od_type="Iter",
        early_stopping_rounds=15000,
        depth=5,
        border_count=64,
        verbose=0
    )
    
    model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=cat_features, use_best_model=True)
    preds = model.predict(X_valid)
    
    f1 = f1_score(y_valid, preds, average='micro')
    fold_scores.append(f1)
    print(f"✅ Fold {fold} F1_micro: {f1:.4f}")

print(f"\n✅ 평균 F1_micro 점수: {np.mean(fold_scores):.4f}")

✅ Fold 1 F1_micro: 0.9104


In [ ]:
# ▶ 5. 전체 데이터로 학습 (최종 모델)
final_model = CatBoostClassifier(
    loss_function="MultiClass",
    eval_metric="MultiClass",
    task_type="GPU",
    learning_rate=0.01,
    iterations=70000,
    l2_leaf_reg=50,
    random_seed=42,
    od_type="Iter",
    depth=5,
    border_count=64,
    verbose=0
)

final_model.fit(X_vif, y_vif, cat_features=cat_features, use_best_model=False)

In [ ]:
# ▶ 6. 모델 저장 (선택사항)
final_model.save_model("피처_40_2/final_catboost_model.cbm")
with open("피처_40_2/label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

# 라벨 인코더도 같이 저장
with open("피처_40_2/label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

# ▶ 7. 테스트 데이터 예측
test = pd.read_csv('피처_40/test_40.csv', encoding='utf-8-sig')

test_id = test['ID']
X_test = test[X_vif.columns]  # 컬럼 순서 맞추기
X_test_pool = Pool(data=X_test, cat_features=cat_features)

preds_class = final_model.predict(X_test_pool)
preds_labels = le.inverse_transform(preds_class.astype(int).flatten())

# ▶ 8. 제출 파일 생성
submission = pd.DataFrame({
    'ID': test_id,
    'Segment': preds_labels
})

# ▶ 9. 저장
submission_path = '피처_40_2/submission_catboost.csv'
submission.to_csv(submission_path, index=False, encoding='utf-8-sig')

print(f"\n📁 제출 파일 저장 완료: {submission_path}")
